In [1]:
import time
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy import stats
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pprint import pprint
import math
from scipy.io import arff
import datetime

In [2]:
# params
seed = 42

Датасет включает в себя данные отчетности банков РФ с 2008 года. Изначально имеется 5 таблиц: 
1. PNL_aggregated.h5, BS_aggregated.h5 содержат группированную отчетность вплоть до самого нижнего уровня группировки (что порождает сотни столбцов). Важно: в этих таблицах имеются столбцы для всех уровней группировки, а значит для каждого банка есть значение И активов, И, к примеру, инвестиций входящих в эти активы.
2. Derevya.xlsx содержит схему группировки для всех периодов для разных уровней (файл для ознакомления, группировка на его основе уже выполнена)
3. BankDefaults.xlsx содержит список дефолтнувших банков и даты дефолтов
4. json-файлы, содержащие словари, дающие списки названий переменных для каждого из уровней дерева: names_levels_bs.json и names_levels_pnl.json


Загрузим данные

In [135]:
path = input()
df_pnl = pd.read_hdf(f"{path}//PNL_aggregated.h5", key = "102")
df_bs = pd.read_hdf(f"{path}//BS_aggregated.h5", key = "102")
df_bs_cbr = pd.read_excel(f"{path}//BS_CBR_aggregated.xlsx")

 C:\\Users\\Ivand\\Repos\\banks\\data


Загрузим словари с уровнями переменных

In [136]:
with open(f"{path}//names_levels_pnl.json", encoding="utf-8") as f:
    names_levels_pnl = json.load(f)
with open(f"{path}//names_levels_bs.json", encoding="utf-8") as f:
    names_levels_bs = json.load(f)
with open(f"{path}//names_levels_bs_cbr.json", encoding="utf-8") as f:
    names_levels_bs_cbr = json.load(f)

In [137]:
names_levels_bs["CBR"] = names_levels_bs_cbr

In [138]:
df_bs_full = df_bs.merge(df_bs_cbr, on = ["DT", "REGN"], how = "outer")

In [139]:
df_bs_full['Cредства клиентов в расчетах']

0         676569.0
1              1.0
2            482.0
3           6664.0
4              5.0
            ...   
125735      5701.0
125736         NaN
125737         NaN
125738         NaN
125739         NaN
Name: Cредства клиентов в расчетах, Length: 125740, dtype: float64

In [7]:
names_levels_bs["CBR"]

['Cредства клиентов в расчетах',
 'Безвозмездное финансирование',
 'Векселя (с учетом переоценки и корректировки стоимости)',
 'Векселя и банковские акцепты',
 'Вложения в ценные бумаги Банка России',
 'Государственные структуры',
 'Дебиторы',
 'Денежные средства (касса, чеки, денежные средства в пути, в банкоматах)',
 'Депозиты в Банке России',
 'Депозиты и прочие привлеченные средства',
 'Дивиденды начисленные',
 'Долговые ценные бумаги',
 'Долговые ценные бумаги, переданные без прекращения признания',
 'Долевые ценные бумаги (с учетом переоценки и изменений справедливой стоимости при первоначальном признании)',
 'Драгоценные металлы и камни',
 'Индивидуальные предприниматели',
 'Корректировка резерва на возможные потери по МСФО 9',
 'Корректировка резервов на возможные потери по МСФО 9',
 'Корреспондентские счета',
 'Корреспондентские счета в Банке России',
 'Корреспондентские счета в кредитных организациях',
 'Кредиторы',
 'Кредиты от Банка России',
 'Кредиты, депозиты и прочие при

Загрузим данные по дефолтам

In [140]:
df_defaults = pd.read_excel(f"{path}//BankDefaults.xlsx")
df_defaults = df_defaults[["BankDefaultIndex", "Name", "regnum", "DefaultType", "DefaultDate","BankLocalization"]]
df_defaults = df_defaults.rename(columns = {"regnum" : "REGN"})
df_defaults

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,BankDefaultIndex,Name,REGN,DefaultType,DefaultDate,BankLocalization
0,1,Финчер,3486-К,ликв.,24.09.2021,Москва
1,2,Платина,2347,отозв.,17.09.2021,Москва
2,3,КС Банк,1752,отозв.,06.08.2021,Саранск (Республика Мордовия)
3,4,Руна-Банк,3207,отозв.,23.07.2021,Москва
4,5,Русское Финансовое Общество,3427-К,отозв.,23.07.2021,Москва
...,...,...,...,...,...,...
2784,2785,Сасовобанк,862,ликв.,28.12.1991,"Сасово, Рязанская обл."
2785,2786,Михайловский,893,ликв.,28.12.1991,Unknown
2786,2787,Мариинско-Посадский Коммерческий Банк,1021,ликв.,28.12.1991,"Мариинский Посад, Чувашия"
2787,2788,"Конверсия, Реконструкция и Развитие",472,отозв.,11.07.1991,Москва


In [19]:
import re
re.sub("-\D+", "","34t53-Л")

'34t53'

In [20]:
#merge with defaults
def func1(x):
    if type(x) == type("str"):
        return re.sub("-\D+", "", x)
    else:
        return x
    
def func2(x):
    if type(x) == type("str"):
        return datetime.datetime.strptime(str(x), "%d.%m.%Y") 
    else:
        return x


pd.options.mode.chained_assignment = None
df_defaults = df_defaults[df_defaults.DefaultType != "ликв."]
df_defaults.DefaultDate = df_defaults.DefaultDate.apply(func2)
df_defaults.REGN = df_defaults.REGN.apply(func1)


In [23]:
df_defaults.to_excel(path + "\\DefaultsWithDates.xlsx")

Напишем функцию, которая будет брать нужные нам уровни в обоих таблицах и мёрджить их

In [24]:
class create_name_masks_container():
    
    def __init__(self, names_levels_pnl, names_levels_bs, pnl, bs, pnl_level, bs_level,CBR = True, additional_variables = []):
    #currently not all the names in the dictionary are real columns
    #luckily, not much of them
    #if I will ever fix this bug, I will deprecate the code below
        if (hasattr(pnl_level, '__len__') == False):
            self.pnl_mask = [i for i in np.array(names_levels_pnl[str(pnl_level)]) if i in np.array(pnl.columns)]
        else: 
            self.pnl_mask = []
            for level in pnl_level:
                self.pnl_mask = self.pnl_mask + [i for i in np.array(names_levels_pnl[str(level)]) if i in np.array(pnl.columns)]

        if (hasattr(bs_level, '__len__') == False):
            self.bs_mask = [i for i in np.array(names_levels_bs[str(bs_level)]) if i in np.array(bs.columns)]
        else: 
            self.bs_mask = []
            for level in bs_level:
                self.bs_mask = self.bs_mask + [i for i in np.array(names_levels_bs[str(level)]) if i in np.array(bs.columns)]

        if CBR:
            self.bs_cbr_mask = [i for i in np.array(names_levels_bs["CBR"]) if i in np.array(bs.columns)]
            self.bs_cbr_mask_check = [i for i in np.array(names_levels_bs["CBR"])]
        else:
            self.bs_cbr_mask = []
        
        self.pnl_encoding = [f"PNL{str(i)}" for i in range(1, len(self.pnl_mask) + 1)]
        self.bs_encoding = [f"BS{str(i)}" for i in range(1, len(self.bs_mask) + 1)]
        self.bs_cbr_encoding = [f"CBRBS{str(i)}" for i in range(1, len(self.bs_cbr_mask) + 1)]
        print(f"cbr encoding mask is {self.bs_cbr_mask}")

        self.additional_variables = additional_variables
    
    def full_true_mask(self):
        return ["DT", "REGN"] + self.additional_variables + self.pnl_mask + self.bs_mask + self.bs_cbr_mask
    
    def pnl_index_mask(self):
        return ["DT", "REGN"] + self.additional_variables + self.pnl_mask
    
    def bs_index_mask(self):
        return ["DT", "REGN"] + self.additional_variables + self.bs_mask
    
    def bs_cbr_index_mask(self):
        return ["DT", "REGN"] + self.additional_variables + self.bs_cbr_mask
    
    def encoding_mask(self):
        return ["DT", "REGN"] + self.additional_variables + self.pnl_encoding + self.bs_encoding + self.bs_cbr_encoding
    
class create_days_container():
    
    def __init__(self, target_days):
        
        if (hasattr(target_days, '__len__') == False):
            self.target_days = [target_days]
        else:
            self.target_days = target_days
        
        self.target_names = [""]*len(self.target_days)
        for days_index in range(len(self.target_days)):
            self.target_names[days_index] = f"DefaultIn{self.target_days[days_index]}Days"
        
    def compare_days(self, number_of_days_real, number_of_days_benchmark):
        if pd.isnull(number_of_days_real):
            return 0
        else:
            return (int(number_of_days_real.days) <= int(number_of_days_benchmark))*1
        
    def create_target_columns(self, df, days_to_default_column = "DaysToDefault"):
        days_to_default_column = df[days_to_default_column]

        for days, days_name in zip(self.target_days, self.target_names):
            df[days_name] = days_to_default_column.apply(self.compare_days, number_of_days_benchmark = days)
        return df
    
    
def prepare_df_to_modelling(pnl, bs, defaults, name_masks, target_days, fillnan = 0):

    #create bs and pnl of needed level
    restricted_pnl = pnl[name_masks_container.pnl_index_mask()]
    if name_masks.bs_cbr_mask == []:
        restricted_bs = bs[name_masks_container.bs_index_mask()]
    else:
        print(f"bs columns are {bs.columns}")
        restricted_bs = bs[name_masks_container.bs_index_mask() + name_masks_container.bs_cbr_mask]
    
    #count NaN in each line
    restricted_bs["BSNan"] = restricted_bs.isnull().sum(axis = 1)
    restricted_pnl["PNLNan"] = restricted_pnl.isnull().sum(axis = 1)
    if fillnan == fillnan:
        restricted_pnl.fillna(fillnan, inplace = True)
        restricted_bs.fillna(fillnan, inplace = True)
    
    #merge bs, bs_cbr and pnl
    merged_reporting = restricted_bs.merge(restricted_pnl, on = ["DT", "REGN"], how = "outer")
    merged_reporting = merged_reporting[name_masks_container.full_true_mask() + ["PNLNan", "BSNan"]]
    print(merged_reporting.columns)
    print(name_masks_container.encoding_mask() + ["PNLNan", "BSNan"] )
    merged_reporting.columns = name_masks_container.encoding_mask() + ["PNLNan", "BSNan"] 

    #merge with defaults
    def func2(x):
        if type(x) == type("str"):
            return datetime.datetime.strptime(str(x), "%d.%m.%Y") 
        else:
            return x
        
    merged_reporting.REGN = merged_reporting.REGN.apply(str)
    
    pd.options.mode.chained_assignment = None
    defaults = defaults[defaults.DefaultType != "ликв."]
    defaults.DefaultDate = defaults.DefaultDate.apply(func2)
    merged_reporting = merged_reporting.merge(defaults, how = "left", on = "REGN")

    merged_reporting["DaysToDefault"] = merged_reporting.DefaultDate - merged_reporting.DT
    days_container_instance = create_days_container(target_days)

    merged_reporting = days_container_instance.create_target_columns(merged_reporting)
    
    not_needed_columns = np.array(defaults.columns)
    not_needed_columns = not_needed_columns[not_needed_columns != "REGN"]
    merged_reporting.drop(list(not_needed_columns) + ["DaysToDefault"], axis = 1, inplace = True)
    
    merged_reporting["Year"] = merged_reporting.DT.apply(lambda x: x.year)
    merged_reporting["Month"] = merged_reporting.DT.apply(lambda x: x.month)
    
    merged_reporting = merged_reporting.sort_values(by = ["REGN", "DT"])
    merged_reporting.fillna(method = "ffill", inplace = True)
    return merged_reporting


In [23]:
name_masks_container = create_name_masks_container(names_levels_pnl, names_levels_bs, df_pnl, df_bs_full, [1, 5], [1, 2], CBR = True)
df = prepare_df_to_modelling(df_pnl, df_bs_full, df_defaults, name_masks_container, [365, 365*2, 10000])


cbr encoding mask is ['Cредства клиентов в расчетах', 'Безвозмездное финансирование', 'Векселя (с учетом переоценки и корректировки стоимости)', 'Векселя и банковские акцепты', 'Вложения в ценные бумаги Банка России', 'Государственные структуры', 'Дебиторы', 'Денежные средства (касса, чеки, денежные средства в пути, в банкоматах)', 'Депозиты в Банке России', 'Депозиты и прочие привлеченные средства', 'Дивиденды начисленные', 'Долговые ценные бумаги', 'Долговые ценные бумаги, переданные без прекращения признания', 'Долевые ценные бумаги (с учетом переоценки и изменений справедливой стоимости при первоначальном признании)', 'Драгоценные металлы и камни', 'Индивидуальные предприниматели', 'Корректировка резерва на возможные потери по МСФО 9', 'Корректировка резервов на возможные потери по МСФО 9', 'Корреспондентские счета', 'Корреспондентские счета в Банке России', 'Корреспондентские счета в кредитных организациях', 'Кредиторы', 'Кредиты от Банка России', 'Кредиты, депозиты и прочие привл

<ipython-input-22-141207ebdb37>:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restricted_bs["BSNan"] = restricted_bs.isnull().sum(axis = 1)
<ipython-input-22-141207ebdb37>:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restricted_pnl["PNLNan"] = restricted_pnl.isnull().sum(axis = 1)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata

Index(['DT', 'REGN', 'Чистая прибыль /(убыток)',
       'Чистый процент.доход/(убыток) до формирования/(возмещ) РВПС',
       'Возмещение /(формирование) резервов на потери по ссудам, ценным бумагам и финансовому лизингу',
       'Чистые непроцентные доходы по операциям с ценными бумагами',
       'Чистые непроцентные доходы по операциям с ПФИ, валютой, драг.металлами и драг. Камнями',
       'Чистые комиссионные доходы', 'Чистые доходы от инвестиций',
       'Чистый доход по операциям сдечи имущества в аренду (в т.ч. лизинга)',
       ...
       'Требования по начисленным процентам (без учета начисленных процентов (купонов) по ценным бумагам)',
       'Уставный капитал кредитных организаций',
       'Участие в уставных капиталах дочерних и зависимых акционерных обществах, паевых инвестиционных фондах',
       'Физические лица', 'Финансовая аренда (лизинг)',
       'Финансовые организации', 'Чистая прибыль текущего года',
       'Эмисcионный доход', 'PNLNan', 'BSNan'],
      dtype='obj

In [24]:
df

,DT,REGN,PNL1,PNL2,PNL3,PNL4,PNL5,PNL6,PNL7,PNL8,...,CBRBS65,CBRBS66,CBRBS67,PNLNan,BSNan,DefaultIn365Days,DefaultIn730Days,DefaultIn10000Days,Year,Month
0,2007-01-01,1,-5439182.0,-7554951.0,1811692.0,194368.0,-1419535.0,-2369408.0,-14492.0,-4699.0,...,60201.0,0.0,7837.0,2.0,36.0,0,0,0,2007,1
409,2007-02-01,1,-5439182.0,-7554951.0,1811692.0,194368.0,-1419535.0,-2369408.0,-14492.0,-4699.0,...,9857.0,0.0,7837.0,2.0,36.0,0,0,0,2007,2
995,2007-03-01,1,-5439182.0,-7554951.0,1811692.0,194368.0,-1419535.0,-2369408.0,-14492.0,-4699.0,...,144490.0,0.0,7837.0,2.0,35.0,0,0,0,2007,3
1629,2007-04-01,1,-2483479.0,-2417485.0,-133707.0,89290.0,-380237.0,-578741.0,0.0,0.0,...,289235.0,0.0,7837.0,2.0,35.0,0,0,0,2007,4
2279,2007-05-01,1,-2483479.0,-2417485.0,-133707.0,89290.0,-380237.0,-578741.0,0.0,0.0,...,287493.0,0.0,7837.0,2.0,33.0,0,0,0,2007,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95743,2016-08-01,992,-51261.0,-17486.0,53099.0,-85340.0,-66247.0,-8122.0,0.0,-9893.0,...,29625.0,-43209.0,0.0,1.0,37.0,1,1,1,2016,8
96383,2016-09-01,992,-51261.0,-17486.0,53099.0,-85340.0,-66247.0,-8122.0,0.0,-9893.0,...,38625.0,21540.0,0.0,1.0,37.0,1,1,1,2016,9
97010,2016-10-01,992,-37506.0,47470.0,61726.0,-156479.0,-84845.0,-13325.0,14.0,-14666.0,...,38625.0,47441.0,0.0,0.0,37.0,1,1,1,2016,10
97632,2016-11-01,992,-37506.0,47470.0,61726.0,-156479.0,-84845.0,-13325.0,14.0,-14666.0,...,38625.0,4938.0,0.0,0.0,38.0,1,1,1,2016,11


In [74]:
df.to_csv(path + "\\BanksDF.csv")

### Список факторов:


In [75]:
factors_dict = pd.DataFrame({"Name" : name_masks_container.full_true_mask() + ["Year", "Month"], 
              "Attribute" : name_masks_container.encoding_mask()+ ["Year", "Month"]})
factors_dict

,Name,Attribute
0,DT,DT
1,REGN,REGN
2,Чистая прибыль /(убыток),PNL1
3,Чистый процент.доход/(убыток) до формирования/...,PNL2
4,Возмещение /(формирование) резервов на потери ...,PNL3
...,...,...
108,Финансовые организации,CBRBS65
109,Чистая прибыль текущего года,CBRBS66
110,Эмисcионный доход,CBRBS67
111,Year,Year


Для единообразия нотации, создадим новую зависимую переменную target:

In [76]:
df.rename(columns = {"class":"target"}, inplace = True)

Пропуски заполнены NaN:

In [77]:
df[df.PNL3.isnull()]

,DT,REGN,PNL1,PNL2,PNL3,PNL4,PNL5,PNL6,PNL7,PNL8,...,CBRBS65,CBRBS66,CBRBS67,PNLNan,BSNan,DefaultIn365Days,DefaultIn730Days,DefaultIn10000Days,Year,Month


dfВыделяем test&train сеты

In [78]:
df

,DT,REGN,PNL1,PNL2,PNL3,PNL4,PNL5,PNL6,PNL7,PNL8,...,CBRBS65,CBRBS66,CBRBS67,PNLNan,BSNan,DefaultIn365Days,DefaultIn730Days,DefaultIn10000Days,Year,Month
0,2007-01-01,1,-5439182.0,-7554951.0,1811692.0,194368.0,-1419535.0,-2369408.0,-14492.0,-4699.0,...,60201.0,0.0,7837.0,2.0,36.0,0,0,0,2007,1
409,2007-02-01,1,-5439182.0,-7554951.0,1811692.0,194368.0,-1419535.0,-2369408.0,-14492.0,-4699.0,...,9857.0,0.0,7837.0,2.0,36.0,0,0,0,2007,2
995,2007-03-01,1,-5439182.0,-7554951.0,1811692.0,194368.0,-1419535.0,-2369408.0,-14492.0,-4699.0,...,144490.0,0.0,7837.0,2.0,35.0,0,0,0,2007,3
1629,2007-04-01,1,-2483479.0,-2417485.0,-133707.0,89290.0,-380237.0,-578741.0,0.0,0.0,...,289235.0,0.0,7837.0,2.0,35.0,0,0,0,2007,4
2279,2007-05-01,1,-2483479.0,-2417485.0,-133707.0,89290.0,-380237.0,-578741.0,0.0,0.0,...,287493.0,0.0,7837.0,2.0,33.0,0,0,0,2007,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95743,2016-08-01,992,-51261.0,-17486.0,53099.0,-85340.0,-66247.0,-8122.0,0.0,-9893.0,...,29625.0,-43209.0,0.0,1.0,37.0,1,1,1,2016,8
96383,2016-09-01,992,-51261.0,-17486.0,53099.0,-85340.0,-66247.0,-8122.0,0.0,-9893.0,...,38625.0,21540.0,0.0,1.0,37.0,1,1,1,2016,9
97010,2016-10-01,992,-37506.0,47470.0,61726.0,-156479.0,-84845.0,-13325.0,14.0,-14666.0,...,38625.0,47441.0,0.0,0.0,37.0,1,1,1,2016,10
97632,2016-11-01,992,-37506.0,47470.0,61726.0,-156479.0,-84845.0,-13325.0,14.0,-14666.0,...,38625.0,4938.0,0.0,0.0,38.0,1,1,1,2016,11


In [79]:
X = df.drop(["DefaultIn365Days", "DefaultIn730Days", "DefaultIn10000Days"], axis = 1)
y = df[["DefaultIn365Days", "DefaultIn730Days", "DefaultIn10000Days"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)


Сохраняем данные

In [ ]:
!pip install pyarrow
!pip install fastparquet

In [80]:
# Save data & info ===
# parquet is optimized for large volumes of data
!mkdir samples
X_train.to_parquet('./samples/X_train.parquet')
X_test.to_parquet('./samples/X_test.parquet')
# переводим pd.Series в pd.DataFrame для удобного экспорта
pd.DataFrame(y_train).to_parquet('./samples/y_train.parquet')
pd.DataFrame(y_test).to_parquet('./samples/y_test.parquet')

#списки категориальных и количественных переменных
df_number_of_uniques = df.nunique()
presumably_continuous = df_number_of_uniques[df_number_of_uniques >= 15]
presumably_discrete = df_number_of_uniques[df_number_of_uniques < 15]

presumably_continuous_names = list(presumably_continuous.index)
presumably_discrete_names = list(presumably_discrete.index)

with open('factors.json', 'w') as f:
    json.dump({'cat_vals': presumably_discrete_names, "num_vals": presumably_continuous_names}, f)

### Статистики

Целевых событий на горизонте года немного, на горизонте двух лет - достаточно прилично, на всем периоде наблюдения - почти половина

In [24]:
print(f'Количество наблюдений: {X.shape[0]}')
print(f'Количество наблюдений, где имеются данные о дефолте или его отсутствии: {X[y.isnull() == False].shape[0]}')
print(f'Количество факторов: {X.shape[1]}')
print(f'Количество целевых событий: {y.sum()}')
print(f'Доля целевых событий: {y.sum() / X[y.isnull() == False].shape[0] * 100}%')

Количество наблюдений: 126035
Количество наблюдений, где имеются данные о дефолте или его отсутствии: 126035
Количество факторов: 45
Количество целевых событий: DefaultIn365Days       6412
DefaultIn730Days      12757
DefaultIn10000Days    48850
dtype: int64
Доля целевых событий: DefaultIn365Days       5.087476
DefaultIn730Days      10.121792
DefaultIn10000Days    38.759075
dtype: float64%


### Пропуски

В датасете есть переменные почти без пропусков, есть с большим количеством. Пропуски есть смысл заполнить нулями, потому что они практически всегда означают, что банк не отчитался о той или иной строке отчетности.

In [317]:
for col in X.columns:
    print(f'Количество пропусков по фактору {col}:\n\t{X[col].isna().sum()} или {X[col].isna().sum() / df.shape[0] * 100}%')

Количество пропусков по фактору DT:
	0 или 0.0%
Количество пропусков по фактору PNL1:
	0 или 0.0%
Количество пропусков по фактору PNL2:
	129 или 0.30586115326251895%
Количество пропусков по фактору PNL3:
	126 или 0.29874810318664646%
Количество пропусков по фактору PNL4:
	3043 или 7.215003793626708%
Количество пропусков по фактору PNL5:
	870 или 2.062784522003035%
Количество пропусков по фактору PNL6:
	6 или 0.014226100151745068%
Количество пропусков по фактору PNL7:
	4904 или 11.627465857359635%
Количество пропусков по фактору PNL8:
	1935 или 4.587917298937785%
Количество пропусков по фактору PNL9:
	93 или 0.22050455235204858%
Количество пропусков по фактору PNL10:
	28148 или 66.73937784522003%
Количество пропусков по фактору PNL11:
	0 или 0.0%
Количество пропусков по фактору PNL12:
	17234 или 40.86210166919575%
Количество пропусков по фактору PNL13:
	4288 или 10.166919575113809%
Количество пропусков по фактору BS1:
	0 или 0.0%
Количество пропусков по фактору BS2:
	0 или 0.0%
Количест

Попробуем реклассифицировать переменные в балансе с третьего уровня наверх

In [107]:
re.sub("\\_[xy]$", "", "sdv_xfsdf")

'sdv_xfsdf'

In [141]:
class create_name_masks_container():
    
    def __init__(self, names_levels_pnl, names_levels_bs, pnl, bs, pnl_level, bs_level,CBR = True, additional_variables = []):
    #currently not all the names in the dictionary are real columns
    #luckily, not much of them
    #if I will ever fix this bug, I will deprecate the code below
        if (hasattr(pnl_level, '__len__') == False):
            self.pnl_mask = [i for i in np.array(names_levels_pnl[str(pnl_level)]) if i in np.array(pnl.columns)]
        else: 
            self.pnl_mask = []
            for level in pnl_level:
                self.pnl_mask = self.pnl_mask + [i for i in np.array(names_levels_pnl[str(level)]) if i in np.array(pnl.columns)]

        if (hasattr(bs_level, '__len__') == False):
            self.bs_mask = [i for i in np.array(names_levels_bs[str(bs_level)]) if i in np.array(bs.columns)]
        else: 
            self.bs_mask = []
            for level in bs_level:
                self.bs_mask = self.bs_mask + [i for i in np.array(names_levels_bs[str(level)]) if i in np.array(bs.columns)]

        if CBR:
            self.bs_cbr_mask = [i for i in np.array(names_levels_bs["CBR"]) if i in np.array(bs.columns)]
            self.bs_cbr_mask_check = [i for i in np.array(names_levels_bs["CBR"])]
        else:
            self.bs_cbr_mask = []
        
        self.pnl_encoding = [f"PNL{str(i)}" for i in range(1, len(self.pnl_mask) + 1)]
        self.bs_encoding = [f"BS{str(i)}" for i in range(1, len(self.bs_mask) + 1)]
        self.bs_cbr_encoding = [f"CBRBS{str(i)}" for i in range(1, len(self.bs_cbr_mask) + 1)]

        self.additional_variables = additional_variables
    
    def full_true_mask(self):
        return ["DT", "REGN"] + self.additional_variables + self.pnl_mask + self.bs_mask + self.bs_cbr_mask
    
    def pnl_index_mask(self):
        return ["DT", "REGN"] + self.additional_variables + self.pnl_mask
    
    def bs_index_mask(self):
        return ["DT", "REGN"] + self.additional_variables + self.bs_mask
    
    def bs_cbr_index_mask(self):
        return ["DT", "REGN"] + self.additional_variables + self.bs_cbr_mask
    
    def encoding_mask(self):
        return ["DT", "REGN"] + self.additional_variables + self.pnl_encoding + self.bs_encoding + self.bs_cbr_encoding
    
class create_days_container():
    
    def __init__(self, target_days):
        
        if (hasattr(target_days, '__len__') == False):
            self.target_days = [target_days]
        else:
            self.target_days = target_days
        
        self.target_names = [""]*len(self.target_days)
        for days_index in range(len(self.target_days)):
            self.target_names[days_index] = f"DefaultIn{self.target_days[days_index]}Days"
        
    def compare_days(self, number_of_days_real, number_of_days_benchmark):
        if pd.isnull(number_of_days_real):
            return 0
        else:
            return (int(number_of_days_real.days) <= int(number_of_days_benchmark))*1
        
    def create_target_columns(self, df, days_to_default_column = "DaysToDefault"):
        days_to_default_column = df[days_to_default_column]

        for days, days_name in zip(self.target_days, self.target_names):
            df[days_name] = days_to_default_column.apply(self.compare_days, number_of_days_benchmark = days)
        return df
    
    
def prepare_df_to_modelling(pnl, bs, defaults, name_masks, target_days, fillnan = 0):

    #create bs and pnl of needed level
    restricted_pnl = pnl[name_masks_container.pnl_index_mask()]
    if name_masks.bs_cbr_mask == []:
        restricted_bs = bs[name_masks_container.bs_index_mask()]
    else:
        restricted_bs = bs[name_masks_container.bs_index_mask() + name_masks_container.bs_cbr_mask]
    
    #count NaN in each line
    restricted_bs["BSNan"] = restricted_bs.isnull().sum(axis = 1)
    restricted_pnl["PNLNan"] = restricted_pnl.isnull().sum(axis = 1)
    if fillnan == fillnan:
        restricted_pnl.fillna(fillnan, inplace = True)
        restricted_bs.fillna(fillnan, inplace = True)
    
    #merge bs, bs_cbr and pnl
    merged_reporting = restricted_bs.merge(restricted_pnl, on = ["DT", "REGN"], how = "outer")
    def func3(x):
        return re.sub("\\_[xy]$", "", x)
    merged_reporting.columns = pd.Series(merged_reporting.columns).apply(func3)
#     print(len(pd.Series(merged_reporting.columns).unique()), len(pd.Series(merged_reporting.columns)))
    merged_reporting =merged_reporting.loc[:,~merged_reporting.columns.duplicated()]
#     print(len(pd.Series(merged_reporting.columns).unique()), len(pd.Series(merged_reporting.columns)))
    merged_reporting = merged_reporting[name_masks_container.full_true_mask() + ["PNLNan", "BSNan"]]
#     print(len(pd.Series(merged_reporting.columns).unique()), len(pd.Series(merged_reporting.columns)))
    merged_reporting.columns = name_masks_container.encoding_mask() + ["PNLNan", "BSNan"] 

    #merge with defaults
    def func2(x):
        if type(x) == type("str"):
            return datetime.datetime.strptime(str(x), "%d.%m.%Y") 
        else:
            return x
        
    merged_reporting.REGN = merged_reporting.REGN.apply(str)
    
    pd.options.mode.chained_assignment = None
    defaults = defaults[defaults.DefaultType != "ликв."]
    defaults.DefaultDate = defaults.DefaultDate.apply(func2)
    merged_reporting = merged_reporting.merge(defaults, how = "left", on = "REGN")

    merged_reporting["DaysToDefault"] = merged_reporting.DefaultDate - merged_reporting.DT
    days_container_instance = create_days_container(target_days)

    merged_reporting = days_container_instance.create_target_columns(merged_reporting)
    
    not_needed_columns = np.array(defaults.columns)
    not_needed_columns = not_needed_columns[not_needed_columns != "REGN"]
    merged_reporting.drop(list(not_needed_columns) + ["DaysToDefault"], axis = 1, inplace = True)
    
    merged_reporting["Year"] = merged_reporting.DT.apply(lambda x: x.year)
    merged_reporting["Month"] = merged_reporting.DT.apply(lambda x: x.month)
    
    merged_reporting = merged_reporting.sort_values(by = ["REGN", "DT"])
    merged_reporting.fillna(method = "ffill", inplace = True)
    return merged_reporting


In [214]:
name_masks_container = create_name_masks_container(names_levels_pnl, names_levels_bs, df_pnl, df_bs_full, [1, 5], [3])
df = prepare_df_to_modelling(df_pnl, df_bs_full, df_defaults, name_masks_container, [365, 365*2, 10000])

factors_dict = pd.DataFrame({"Name" : name_masks_container.full_true_mask() + ["Year", "Month"], 
              "Attribute" : name_masks_container.encoding_mask()+ ["Year", "Month"]})

reclassification_dict = pd.read_excel(path + "\\Classify2.xlsx", header = 1)
reclassification_dict = reclassification_dict[list(reclassification_dict.columns)[0:55]]
reclassification_dict

recl_dict = {}
for i in reclassification_dict.columns:
    names = reclassification_dict[i][reclassification_dict[i].isnull() == False]
    codes = pd.Series(names).map(dict(zip(factors_dict.Name, factors_dict.Attribute)))
    codes = codes[codes.isnull() == False]
    recl_dict[i] = codes

df = df.copy()
for group in list(recl_dict.keys()):
    df[group] = df[recl_dict[group]].sum(axis = 1)
df = df.drop(["Доходы_физики", "Доходы_корп", "Расходы_физики", "Расходы_корп", "Расходы_физики", "Расходы_резервы"], axis = 1)

In [215]:
reclassification_dict

,Резервы,Ленды_корп,Ленды_ритейл,Борроу_корп,Борроу_ритейл,Ленды_межбанк,Борроу_межбанк,Ленды_меньше_года,Борроу_меньше_года,Ленды_больше_года,...,Средства клиентов,Выпущенные долговые ценные бумаги,"Обязательства по производным финансовым инструментам, \nпо которым ожидается уменьшение экономических выгод",Прочие обязательства,"Переоценка, увеличивающая/уменьшающая стоимость обязательств с учетом корректировки по МСФО 9",Уставный капитал и эмиcсионный доход1,Составляющие добавочного капитала,Резервный фонд,"Переоценка ценных бумаг, оцениваемых по справедливой стоимости и резервы на возможные потери",Накопленная прибыль (убыток)
0,Обязательные резервы в ЦБ РФ,"Долговые ценные бумаги корпораций резидентов, ...",Ссуды физ.лицам-предпринимателям до 30 дней,Срочные депозиты корпоративных клиентов-резиде...,Счета физ.лиц-резидентов до востребования и де...,Средства в банках и на биржах до востребования,Средства банков резидентов до 30 дней,Депозиты в ЦБ РФ на срок от 1 до 12 мес,"Кредиты и депозиты, полученные от ЦБ РФ до вос...",Депозиты в ЦБ РФ на срок свыше 1 года,...,Средства корпоративных клиентов,Облигации,Обязательства по производным финансовым инстру...,Резервы – оценочные обязательства некредитного...,"Переоценка, увеличивающая/уменьшающая стоимост...",Уставный капитал кредитных организаций,Составляющие добавочного капитала,Резервный фонд,"Переоценка ценных бумаг, оцениваемых по справе...",Прибыль (убыток) прошлых лет
1,Резервы по средствам в банках до востребования,Долговые ценные бумаги корпораций нерезидентов...,Ссуды физ.лицам-предпринимателям на срок от 1 ...,Счета ИЧП и субъектов малого бизнеса и депозит...,Срочные депозиты ИЧП на срок от 1 до 12 мес,Размещенные МБК и МБД до 30 дней- резиденты,Средства банков нерезидентов до 30 дней,Средства в банках и на биржах до востребования,"Кредиты и депозиты, полученные от ЦБ РФ на сро...",Размещенные МБК и МБД свыше 1 года - резиденты,...,Депозиты и прочие привлеченные средства,Векселя и банковские акцепты,Обязательства по производным финансовым инстру...,Расчеты кредитной организации по отдельным опе...,"Переоценка, увеличивающая/уменьшающая стоимост...",Эмисcионный доход,Составляющие добавочного капитала,Резервный фонд,Резервы на возможные потери (без учета МСФО 9),Чистая прибыль текущего года
2,РВП по МБК и МБД - резиденты,"Долевые ценные бумаги корпораций резидентов, о...",Ссуды физ.лицам-предпринимателям свыше 1 года,Срочные депозиты корпоративных клиентов-резиде...,Срочные депозиты ИЧП на срок свыше 1 года,Размещенные МБК и МБД от 1 до 12 мес. - резиденты,"Счета на биржах, ЛОРО счета банков-резидентов ...",Размещенные МБК и МБД до 30 дней- резиденты,Средства банков резидентов до 30 дней,Размещенные МБК и МБД свыше 1 года - нерезиденты,...,Средства на счетах,NaN,NaN,Кредиторы,NaN,"Собственные доли уставного капитала (акции), в...",NaN,NaN,Корректировка резерва на возможные потери по М...,Дивиденды начисленные
3,РВП по МБК и МБД - нерезиденты,"Долевые ценные бумаги корпораций нерезидентов,...",Ссуды физ.лицам резидентам до 30 дней,Счета физ.лиц до востребования и депозиты до 3...,Срочные депозиты физ.лиц-резидентов на срок от...,Размещенные МБК и МБД свыше 1 года - резиденты,ЛОРО счета банков-нерезидентов и прочие счета ...,Размещенные МБК и МБД от 1 до 12 мес. - резиденты,Средства банков нерезидентов до 30 дней,Ссуды государственным и субфедеральным структу...,...,Государственные средства,NaN,NaN,Отложенное налоговое обязательство,NaN,Безвозмездное финансирование,NaN,NaN,NaN,NaN
4,РВП по просроченным требованиям к банкам,"Векселя корпораций резидентов, оцениваемые по ...",Ссуды физ.лицам резидентам на срок от 1 до 12 мес,Счета корпоративных клиентов-нерезидентов до в...,Срочные депозиты физ.лиц-резидентов на срок св...,Размещенные МБК и МБД до 30 дней - нерезиденты,Средства банков резидентов на срок от 1 до 12 ...,Размещенные МБК и МБД до 30 дней - нерезиденты,Средства банков резидентов на срок от 1 до 12 ...,Ссуды корпоративным клиентам-резидентам свыше ...,...,Депозиты

In [216]:
name_masks_container = create_name_masks_container(names_levels_pnl, names_levels_bs, df_pnl, df_bs_full, [1, 6], [3])
df_sixth = prepare_df_to_modelling(df_pnl, df_bs_full, df_defaults, name_masks_container, [365, 365*2, 10000])
df_sixth = df_sixth[["DT", "REGN","Month", "Year", "PNL2", "PNL8"]]
keys = keys + ["CreditsIncome", "DepositsExpenditure"]
df_sixth.columns = ["DT", "REGN", "Month", "Year","CreditsIncome", "DepositsExpenditure"]
df_sixth

In [217]:
name_masks_container = create_name_masks_container(names_levels_pnl, names_levels_bs, df_pnl, df_bs_full, [7], [3])
df_seventh = prepare_df_to_modelling(df_pnl, df_bs_full, df_defaults, name_masks_container, [365, 365*2, 10000])
factors_dict = pd.DataFrame({"Name" : name_masks_container.full_true_mask() + ["Year", "Month"], 
              "Attribute" : name_masks_container.encoding_mask()+ ["Year", "Month"]})

reclassification_dict = pd.read_excel(path + "\\Classify2.xlsx", header = 1)
reclassification_dict = reclassification_dict[list(reclassification_dict.columns)[0:55]]
reclassification_dict

recl_dict = {}
for i in reclassification_dict.columns:
    names = reclassification_dict[i][reclassification_dict[i].isnull() == False]
    codes = pd.Series(names).map(dict(zip(factors_dict.Name, factors_dict.Attribute)))
    codes = codes[codes.isnull() == False]
    recl_dict[i] = codes

df_seventh = df_seventh.copy()
for group in list(recl_dict.keys()):
    df_seventh[group] = df_seventh[recl_dict[group]].sum(axis = 1)

df_seventh = df_seventh[["DT", "REGN", "Month", "Year", "Доходы_корп", "Доходы_физики", "Расходы_физики", "Расходы_корп", "Расходы_резервы"]]
#df = df.drop(["Доходы_корп", "Доходы_корп", "Расходы_физики", "Расходы_корп", "Расходы_физики", "Расходы_резервы"], axis = 1)
df_seventh

,DT,REGN,Month,Year,Доходы_корп,Доходы_физики,Расходы_физики,Расходы_корп,Расходы_резервы
0,2007-01-01,1,1,2007,-8263819.0,-1492078.0,460836.0,3613294.0,3496713.0
409,2007-02-01,1,2,2007,-8263819.0,-1492078.0,460836.0,3613294.0,3496713.0
995,2007-03-01,1,3,2007,-8263819.0,-1492078.0,460836.0,3613294.0,3496713.0
1629,2007-04-01,1,4,2007,-2692485.0,-597180.0,150938.0,1331600.0,278978.0
2279,2007-05-01,1,5,2007,-2692485.0,-597180.0,150938.0,1331600.0,278978.0
...,...,...,...,...,...,...,...,...,...
95743,2016-08-01,992,8,2016,-64559.0,-10925.0,63759.0,16841.0,96567.0
96383,2016-09-01,992,9,2016,-64559.0,-10925.0,63759.0,16841.0,96567.0
97010,2016-10-01,992,10,2016,-93510.0,-16114.0,99741.0,25962.0,126723.0
97632,2016-11-01,992,11,2016,-93510.0,-16114.0,99741.0,25962.0,126723.0


In [218]:
df_new = df.merge(df_sixth, on = ['REGN', "Month", "Year"])
df_new = df_new.merge(df_seventh, on = ['REGN', "Month", "Year"])

In [219]:
df_reclassified = df_new[list(recl_dict.keys()) + ["DT", "REGN", "Month", "Year", "CreditsIncome", "DepositsExpenditure"]]

In [220]:
name_masks_container = create_name_masks_container(names_levels_pnl, names_levels_bs, df_pnl, df_bs, [1, 5], [1, 2])
df = prepare_df_to_modelling(df_pnl, df_bs, df_defaults, name_masks_container, [365, 365*2, 10000])

In [221]:
df_interesting = df[["REGN", "Year", "Month", "PNL1", "PNL4", "PNL5", "PNL6", "PNL7", "PNL2", "BS1", "BS2", "BS3", "BS5", "BS12", "BS15", "BS18","BS16", "BS29", "DefaultIn365Days", "DefaultIn730Days", "DefaultIn10000Days"]]
df_interesting["ImmobilizedAssets"] = df_interesting.BS15 + df_interesting.BS18
df_interesting.drop(["BS15", "BS18"], axis = 1, inplace = True)
df_interesting

,REGN,Year,Month,PNL1,PNL4,PNL5,PNL6,PNL7,PNL2,BS1,BS2,BS3,BS5,BS12,BS16,BS29,DefaultIn365Days,DefaultIn730Days,DefaultIn10000Days,ImmobilizedAssets
0,1,2007,1,-5439182.0,194368.0,-1419535.0,-2369408.0,-14492.0,-7554951.0,241835809.0,-221153846.0,-20681963.0,3934565.0,0.0,520994.0,-12854441.0,0,0,0,4372961.0
409,1,2007,2,-5439182.0,194368.0,-1419535.0,-2369408.0,-14492.0,-7554951.0,254819532.0,-233166620.0,-21652912.0,4384525.0,0.0,520977.0,-13825390.0,0,0,0,4239594.0
995,1,2007,3,-5439182.0,194368.0,-1419535.0,-2369408.0,-14492.0,-7554951.0,248291422.0,-226357132.0,-21934290.0,4339921.0,0.0,520984.0,-14106768.0,0,0,0,4224386.0
1629,1,2007,4,-2483479.0,89290.0,-380237.0,-578741.0,0.0,-2417485.0,265220418.0,-242244339.0,-22976079.0,4365484.0,0.0,522787.0,-15148557.0,0,0,0,5105437.0
2279,1,2007,5,-2483479.0,89290.0,-380237.0,-578741.0,0.0,-2417485.0,262509086.0,-240158310.0,-22350776.0,4399595.0,0.0,522806.0,-14523254.0,0,0,0,5365910.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95743,992,2016,8,-51261.0,-85340.0,-66247.0,-8122.0,0.0,-17486.0,2201881.0,-1896190.0,-269017.0,8819.0,1177.0,0.0,-208897.0,1,1,1,228970.0
96383,992,2016,9,-51261.0,-85340.0,-66247.0,-8122.0,0.0,-17486.0,2233772.0,-1944479.0,-256044.0,9326.0,11027.0,0.0,-195924.0,1,1,1,238029.0
97010,992,2016,10,-37506.0,-156479.0,-84845.0,-13325.0,14.0,47470.0,2240207.0,-1921335.0,-287180.0,11023.0,19457.0,0.0,-227060.0,1,1,1,236356.0
97632,992,2016,11,-37506.0,-156479.0,-84845.0,-13325.0,14.0,47470.0,2064643.0,-1812298.0,-238967.0,11012.0,22768.0,0.0,-178847.0,1,1,1,235298.0


In [222]:
df_short = df_interesting.merge(df_reclassified, on = ["REGN", "Year", "Month"])

In [212]:
factors_dict = pd.DataFrame({"Name" : name_masks_container.full_true_mask() + ["Year", "Month"], 
              "Attribute" : name_masks_container.encoding_mask()+ ["Year", "Month"]})
factors_dict

,Name,Attribute
0,DT,DT
1,REGN,REGN
2,Чистая прибыль /(убыток),PNL1
3,Чистый процент.доход/(убыток) до формирования/...,PNL2
4,Возмещение /(формирование) резервов на потери ...,PNL3
5,Чистые непроцентные доходы по операциям с ценн...,PNL4
6,"Чистые непроцентные доходы по операциям с ПФИ,...",PNL5
7,Чистые комиссионные доходы,PNL6
8,Чистые доходы от инвестиций,PNL7
9,Чистый доход по операциям сдечи имущества в ар...,PNL8


In [223]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also

#     print(pd.DataFrame({"Real":df_short.columns, "New" : ["REGN", "Year", "Month", "NetIncome",  "NetSecurityIncome", "NetDerivativeIncome", 
#                    "NetFeeIncome", "NetInvestmentIncome", "NetInterestIncome", "Assets", "Liabilities", "Equity","RequiredReserves", "Securities",
#                    "Investment", "RetainedEarning", 'DefaultIn365Days', 'DefaultIn730Days', 'DefaultIn10000Days',"ImmobilizedAssets", "Reserves_Me", "LendCorporate_Me", "LendRetail", "BorrowCorp_Me", "BorrowRetail_Me",
#                    "LendInterbank_Me", "BorrowInterbank_Me", "LendShort_Me", "BorrowShort_Me", "LendLong_Me", "BorrowLong_Me", 
#                    "Cash_Me", "BondsIssued_Me", "BadCredits_Me", "LendState_Me", "BorrowState_Me", "CurrentAccounts_Me", "TradingAccounts_Me", "BadBorrowing_Me", "ReservesCorrected_Me", 
#                     "Deposits_Me", "Credits_Me", "IncomeRetail_Me", "IncomeCorporates_Me", "ExpenditureRetail_Me", 'ExpenditureCorporates_Me', 'ExpendituresReserves_Me',
#                     'DepositsRetail_Me', 'DepositsCorporates_Me', 'CreditsCorporates_Me', 'CreditsRetail_Me',
#                    "Unclassified", "CBRDeposits_CBR","ObligatoryReserves_CBR","CreditsToBanks_CBR","Securities_CBR","ShareCapitalParticipation_CBR",
#                     "CreditPortfolio_CBR","ProfitableDerivatives_CBR","CoreFunds_CBR","Intangibles_CBR","DeferredTaxAsset_CBR","OtherAssets_CBR",
#                     "CBRCredits_CBR","BanksFunds_CBR","ClientFunds_CBR","SecuritiesIssued_CBR","DerivativeLiabilities_CBR","OtherLiabilities_CBR",
#                     "Revaluation_CBR","MainEquity_CBR","ExtraEquity_CBR","ReserveFund_CBR","SecuritiesRevaluation_CBR", "RetainedEarnings_CBR", "Date", "CreditsIncome", "DepositsExpenditure"
#                     ]}))

In [224]:
df_short.columns

Index(['REGN', 'Year', 'Month', 'PNL1', 'PNL4', 'PNL5', 'PNL6', 'PNL7', 'PNL2',
       'BS1', 'BS2', 'BS3', 'BS5', 'BS12', 'BS16', 'BS29', 'DefaultIn365Days',
       'DefaultIn730Days', 'DefaultIn10000Days', 'ImmobilizedAssets',
       'Резервы', 'Ленды_корп', 'Ленды_ритейл', 'Борроу_корп', 'Борроу_ритейл',
       'Ленды_межбанк', 'Борроу_межбанк', 'Ленды_меньше_года',
       'Борроу_меньше_года', 'Ленды_больше_года', 'Борроу_больше_года', 'Кэш',
       'Бонды_выпущенные', 'Кредиты_просроченные', 'Ленды_госво',
       'Борроу_госво', 'Текущие', 'Брокеры', 'Займы_просроченные',
       'Резервы_корр', 'Депозиты', 'Кредиты', 'Доходы_физики', 'Доходы_корп',
       'Расходы_физики', 'Расходы_корп', 'Расходы_резервы', 'Депозиты_физики',
       'Депозиты_корп', 'Кредиты_корп', 'Кредиты_физики',
       'Не вошло ни в одну категорию', 'Депозиты в Банке России',
       'Обязательные резервы в Банке России ', 'Кредиты банкам',
       'Ценные бумаги', 'Участие в уставных капиталах',
       'Кредит

In [225]:
df_short.columns = ["REGN", "Year", "Month", "NetIncome",  "NetSecurityIncome", "NetDerivativeIncome", 
                   "NetFeeIncome", "NetInvestmentIncome", "NetInterestIncome", "Assets", "Liabilities", "Equity","RequiredReserves", "Securities",
                   "Investment", "RetainedEarning", 'DefaultIn365Days', 'DefaultIn730Days', 'DefaultIn10000Days',"ImmobilizedAssets", "Reserves_Me", "LendCorporate_Me", "LendRetail", "BorrowCorp_Me", "BorrowRetail_Me",
                   "LendInterbank_Me", "BorrowInterbank_Me", "LendShort_Me", "BorrowShort_Me", "LendLong_Me", "BorrowLong_Me", 
                   "Cash_Me", "BondsIssued_Me", "BadCredits_Me", "LendState_Me", "BorrowState_Me", "CurrentAccounts_Me", "TradingAccounts_Me", "BadBorrowing_Me", "ReservesCorrected_Me", 
                    "Deposits_Me", "Credits_Me", "IncomeRetail_Me", "IncomeCorporates_Me", "ExpenditureRetail_Me", 'ExpenditureCorporates_Me', 'ExpendituresReserves_Me',
                    'DepositsRetail_Me', 'DepositsCorporates_Me', 'CreditsCorporates_Me', 'CreditsRetail_Me',
                   "Unclassified", "CBRDeposits_CBR","ObligatoryReserves_CBR","CreditsToBanks_CBR","Securities_CBR","ShareCapitalParticipation_CBR",
                    "CreditPortfolio_CBR","ProfitableDerivatives_CBR","CoreFunds_CBR","Intangibles_CBR","DeferredTaxAsset_CBR","OtherAssets_CBR",
                    "CBRCredits_CBR","BanksFunds_CBR","ClientFunds_CBR","SecuritiesIssued_CBR","DerivativeLiabilities_CBR","OtherLiabilities_CBR",
                    "Revaluation_CBR","MainEquity_CBR","ExtraEquity_CBR","ReserveFund_CBR","SecuritiesRevaluation_CBR","RetainedEarnings_CBR", "Date", "CreditsIncome", "DepositsExpenditure"
                    ]

In [226]:
df_short.NetIncome = -df_short.NetIncome
df_short.NetSecurityIncome = -df_short.NetSecurityIncome
df_short.NetDerivativeIncome = -df_short.NetDerivativeIncome
df_short.NetFeeIncome = -df_short.NetFeeIncome
df_short.NetInvestmentIncome = -df_short.NetInvestmentIncome
df_short.NetInterestIncome = -df_short.NetInterestIncome
df_short.IncomeRetail_Me = -df_short.IncomeRetail_Me
df_short.IncomeCorporates_Me = -df_short.IncomeCorporates_Me
df_short.ExpenditureRetail_Me = -df_short.ExpenditureRetail_Me
df_short.ExpenditureCorporates_Me = -df_short.ExpenditureCorporates_Me
df_short.ExpendituresReserves_Me = -df_short.ExpendituresReserves_Me

In [227]:
df_short.columns

Index(['REGN', 'Year', 'Month', 'NetIncome', 'NetSecurityIncome',
       'NetDerivativeIncome', 'NetFeeIncome', 'NetInvestmentIncome',
       'NetInterestIncome', 'Assets', 'Liabilities', 'Equity',
       'RequiredReserves', 'Securities', 'Investment', 'RetainedEarning',
       'DefaultIn365Days', 'DefaultIn730Days', 'DefaultIn10000Days',
       'ImmobilizedAssets', 'Reserves_Me', 'LendCorporate_Me', 'LendRetail',
       'BorrowCorp_Me', 'BorrowRetail_Me', 'LendInterbank_Me',
       'BorrowInterbank_Me', 'LendShort_Me', 'BorrowShort_Me', 'LendLong_Me',
       'BorrowLong_Me', 'Cash_Me', 'BondsIssued_Me', 'BadCredits_Me',
       'LendState_Me', 'BorrowState_Me', 'CurrentAccounts_Me',
       'TradingAccounts_Me', 'BadBorrowing_Me', 'ReservesCorrected_Me',
       'Deposits_Me', 'Credits_Me', 'IncomeRetail_Me', 'IncomeCorporates_Me',
       'ExpenditureRetail_Me', 'ExpenditureCorporates_Me',
       'ExpendituresReserves_Me', 'DepositsRetail_Me', 'DepositsCorporates_Me',
       'CreditsCorp

In [229]:
df_short.to_csv(path + "\\BanksDFUsable.csv")

### Выводы

Достаточно большой датасет с большим количеством переменных (при желании можно получить больше 300 факторов). К сожалению сами по себе далеко не все факторы глубоко осмыслены, их необходимо группировать в соответствии с логикой сектора. Хорошо то, что датасет расширяем, т.к. не обезличен и то, что у нас есть дата дефолта, позволяющая получить данные о дефолте для разных срочностей.